In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from tqdm import tqdm

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.losses.pytorch import QuantileLoss, DistributionLoss

In [2]:
df = pd.read_csv('cropland_fires.csv')

In [3]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
def prepare_co2_data(df):
    # Filter for CO2 emissions data
    co2_df = df[df['gas'] == 'co2'].copy()
    
    # Convert 'start_time' to datetime if it isn't already
    co2_df['date'] = pd.to_datetime(co2_df['start_time'])
    
    # Create monthly averages
    monthly_avg = (co2_df.groupby(co2_df['date'].dt.strftime('%Y-%m'))
                  ['emissions_quantity']
                  .mean()
                  .reset_index())
    
    # Convert the month strings back to datetime for proper sorting
    monthly_avg['date'] = pd.to_datetime(monthly_avg['date'] + '-01')
    
    # Format data for NeuralForecast
    ts_data = pd.DataFrame({
        'unique_id': 'co2 emissions',
        'ds': monthly_avg['date'],
        'y': monthly_avg['emissions_quantity']
    }).sort_values('ds')
    
    return ts_data

In [5]:
def calculate_forecast_horizon(data, target_year):
    """Calculate the number of months to forecast based on the target year"""
    last_date = pd.to_datetime(data['ds'].max())
    target_date = pd.to_datetime(f'{target_year}-12-31')
    months_diff = ((target_date.year - last_date.year) * 12 + 
                  (target_date.month - last_date.month))
    return max(1, months_diff)

In [6]:
def train_nbeats_model(data, forecast_horizon=2):
    # Initialize NBEATS model
    model = NBEATS(
        input_size=12,          # Use 12 months of history
        h=forecast_horizon,     # Forecast horizon
        batch_size=32,
        learning_rate=1e-3,
        loss=DistributionLoss(distribution='Poisson', level=[80, 90]),           # Mean Squared Error loss
        stack_types = ['identity', 'trend', 'seasonality'],
        n_blocks=[3, 3, 3],
        max_steps=100,
        scaler_type='standard',  # Standardize the data
        random_seed=42
    )

    forecaster = NeuralForecast(
        models=[model],
        freq='M',  # Monthly frequency
    )
    forecaster.fit(data)
    return forecaster

In [7]:
def forecast_co2_emissions(data, target_year):
    # Calculate forecast horizon
    forecast_horizon = calculate_forecast_horizon(data, target_year)
    
    # Initialize N-BEATS model with CPU device
    nbeats = NBEATS(
        h=forecast_horizon,
        input_size=12,
        max_steps=100,
        batch_size=32,
        learning_rate=1e-3,
        stack_types=['trend', 'seasonality'],
        n_blocks=[1, 1],
        n_harmonics=2,
        n_polynomials=2,
        random_seed=42,
    )

    # Create and fit NeuralForecast model
    nf = NeuralForecast(
        models=[nbeats],
        freq='M'
    )
    nf.fit(data)

    # Generate forecast
    forecast = nf.predict()

    # For metrics, use the last available actual values (if any overlap exists)
    actual = data['y'].iloc[-min(forecast_horizon, len(data)):]
    predicted = forecast['NBEATS'].iloc[:len(actual)]

    if len(actual) > 0:
        mae = mean_absolute_error(actual, predicted)
        rmse = np.sqrt(mean_squared_error(actual, predicted))
    else:
        mae = rmse = np.nan

    # Create visualization
    fig = go.Figure()

    # Plot historical data
    fig.add_trace(go.Scatter(
        x=data['ds'], 
        y=data['y'], 
        mode='lines', 
        name='Historical', 
        line=dict(color='blue')
    ))

    # Generate dates for forecast
    forecast_dates = pd.date_range(
        start=data['ds'].iloc[-1] + pd.DateOffset(months=1),
        periods=forecast_horizon,
        freq='M'
    )

    # Plot forecasted values
    fig.add_trace(go.Scatter(
        x=forecast_dates, 
        y=forecast['NBEATS'], 
        mode='lines', 
        name='Forecast', 
        line=dict(color='red')
    ))

    fig.update_layout(
        title='CO2 Emissions Forecast',
        xaxis_title='Date',
        yaxis_title='CO2 Emissions',
        legend=dict(title='Legend'),
        template='plotly_white'
    )
    fig.update_xaxes(tickangle=45)

    fig.show()

    return forecast, mae, rmse, nf


In [8]:
def run_co2_forecast(df, target_year=2023):
    """
    Main function to run the CO2 emissions forecast
    """
    ts_data = prepare_co2_data(df)
    forecast, mae, rmse, model = forecast_co2_emissions(ts_data, target_year)
    
    if not np.isnan(mae):
        print(f"Mean Absolute Error: {mae:.2f}")
        print(f"Root Mean Square Error: {rmse:.2f}")
    else:
        print("No overlap between forecast period and actual data for error calculation")
    
    return forecast, ts_data, mae, rmse, model

In [9]:
forecast, historical_data, mae, rmse, model = run_co2_forecast(df, target_year=2023)

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 1.6 M  | train
-------------------------------------------------------
1.6 M     Trainable params
1.8 K     Non-trainable params
1.6 M     Total params
6.565     Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
/home/hem/personal/clg/sem5/ml/cropfires/.venv/lib/python3.12/site-packages/utilsforecast/processing.py:384: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/hem/personal/clg/sem5/ml/cropfires/.venv/lib/python3.12/site-packages/utilsforecast/processing.py:438: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/hem/personal/clg/sem5/ml/cropfires/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
/tmp/ipykernel_5988/681835571.py:52: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Mean Absolute Error: 147.87
Root Mean Square Error: 177.69
